In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
df = pd.read_hdf('../act_4.h5')
df.head().append(df.tail())

,item_id,course_id,video_id,watching_count,video_duration,local_watching_time,video_progress_time,video_start_time,video_end_time,local_start_time,local_end_time,drop
0,U_7140103,TsinghuaX+20430064X+sp,V_05f095f52ac6446ba05b883171c66713,1,563.0,563,563.000000,0.000,563.000000,2017-07-24 13:26:56,2017-07-24 13:36:19,0
1,U_7140103,TsinghuaX+34000888X+sp,V_0d166f12024942fca500891c0346b86a,1,166.0,166,166.000000,0.000,166.000000,2017-07-06 14:47:00,2017-07-06 14:49:46,0
2,U_7140103,TsinghuaX+00691153X+sp,V_0d2ed7b0f77647388d3ad5420d1d6549,2,415.0,415,414.809998,0.000,415.000000,2017-07-13 09:01:58,2017-07-13 13:18:15,0
3,U_7140103,TsinghuaX+20430064X+sp,V_10355a6a239c4a1dbc28e99bfb7633c1,1,173.0,174,173.000000,0.000,173.000000,2017-07-24 13:11:47,2017-07-24 13:14:41,0
4,U_7140103,TsinghuaX+34000888X+sp,V_11f15f23fca948808b5f44e25b75f6a2,1,63.0,19,18.820000,0.000,18.820000,2017-07-06 14:56:18,2017-07-06 14:56:37,0
128,U_2664891,XJTU+C00204+sp,V_xjtu-C00204-8-1-1,9,591.0,280,325.726007,0.000,317.743011,2017-10-26 22:12:29,2017-10-26 22:17:42,0
129,U_2664891,XJTU+C00204+sp,V_xjtu-C00204-8-1-2,2,665.0,535,666.944000,0.256,665.000000,2017-10-26 22:17:00,2017-10-26 22:30:02,0
130,U_2664891,XJTU+C00204+sp,V_xjtu-C00204-8-1-3,1,603.0,484,602.291000,0.709,603.000000,2017-10-26 22:30:03,2017-10-26 22:38:07,0
131,U_2664891,XJTU+C00204+sp,V_xjtu-C00204-8-1-4,1,670.0,537,669.499000,0.501,670.000000,2017-10-26 22:38:08,2017-10-26 22:47:05,0
132,U_2664891,XJTU+C00204+sp,V_xjtu-C00204-8-2-5,2,606.0,147,180.903004,0.000,178.542999,2017-10-26 22:47:11,2017-10-26 22:49:40,1


In [3]:
df.shape

(730734, 12)

In [4]:
df.drop_duplicates(['item_id', 'course_id', 'video_id']).shape

(727712, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730734 entries, 0 to 132
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   item_id              730734 non-null  object 
 1   course_id            730734 non-null  object 
 2   video_id             730734 non-null  object 
 3   watching_count       730734 non-null  int64  
 4   video_duration       730734 non-null  float64
 5   local_watching_time  730734 non-null  int64  
 6   video_progress_time  730734 non-null  float64
 7   video_start_time     730734 non-null  float64
 8   video_end_time       730734 non-null  float64
 9   local_start_time     730734 non-null  object 
 10  local_end_time       730734 non-null  object 
 11  drop                 730734 non-null  int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 72.5+ MB


In [6]:
df['local_start_time'] = pd.to_datetime(df['local_start_time'])
df['local_end_time'] = pd.to_datetime(df['local_end_time'])
gc.collect()

0

In [7]:
uid1 = ['item_id', 'course_id', 'video_id']
uid2 = ['item_id', 'course_id']

In [8]:
data = df.groupby(uid1)['drop'].apply(list)
data = data.reset_index()
data['drop'] = data['drop'].map(lambda x: 1 if 1 in x else 0)
gc.collect()


# video_start_time, video_end_time, local_start_time, local_end_time
tmp = df.groupby(uid1, as_index=False).agg({
    'watching_count': np.sum,
    'video_duration': np.max,
    'local_watching_time': np.sum,
    'video_progress_time': np.sum,
    'video_start_time': np.min,
    'video_end_time': np.max,
    'local_start_time': np.min,
    'local_end_time': np.max
})
data = data.merge(tmp, on=uid1, how='left')
del tmp
gc.collect()

0

In [9]:
data.head()

,item_id,course_id,video_id,drop,watching_count,video_duration,local_watching_time,video_progress_time,video_start_time,video_end_time,local_start_time,local_end_time
0,U_1000763,HUBU+2017022703X+sp,V_1868fcc69aae4c84873693ab9625e970,0,3,491.0,510,506.982999,0.0,491.0,2017-10-16 11:01:57,2017-10-25 17:20:07
1,U_1000763,HUBU+2017022703X+sp,V_2a08be8f69064c7ebb7951d07959ff12,0,34,251.0,527,493.441940,0.0,251.0,2017-10-25 17:50:44,2018-04-09 15:14:54
2,U_1000763,HUBU+2017022703X+sp,V_31b0a46fe4b74381b55123228091b138,0,3,282.0,72,72.160000,0.0,60.0,2017-10-25 17:51:25,2018-04-09 15:15:55
3,U_1000763,HUBU+2017022703X+sp,V_3f4d13f054974818948f6783c1a5bbfd,0,1,394.0,396,394.000000,0.0,394.0,2017-10-16 10:55:20,2017-10-16 11:01:56
4,U_1000763,HUBU+2017022703X+sp,V_7c71f2f189274698a773329684114ae5,0,54,395.0,1141,1138.059989,0.0,395.0,2017-10-25 17:44:18,2017-11-06 16:27:38


In [10]:
data.shape

(727712, 12)

In [11]:
# 最晚的进度条时间点-最早的进度条时间点
data['video_watching_duration'] = data['video_end_time'] - data['video_start_time']

# 观看结束时间-观看开始时间
data['local_watching_duration'] = data.apply(lambda x: (x['local_end_time'] - x['local_start_time']).seconds, axis=1)

In [12]:
# 进度条是否读到最后，小于0
data['video_is_end'] = data['video_end_time'] - data['video_duration']

# 一开始是否拖到进度条
data['video_is_start'] = data['video_start_time'].map(lambda x: 1 if x > 0 else 0)

# 进度条是否从开始读到最后，小于0
data['video_is_start_end'] = data['video_watching_duration'] - data['video_duration']

# 进度条拖动的秒速 - 暂停时长
data['forward_seconds'] = data['video_watching_duration'] - data['video_progress_time']

# 暂停时长，
# df[''] = df[''] - df['']

# 实际观看时长是否有差误
data['local_watching_diff'] = data['local_watching_duration'] - data['local_watching_time']

# 倍速
data['speed'] = data['local_watching_time'] / (data['video_progress_time'] + 0.001)

In [13]:
data.columns

Index(['item_id', 'course_id', 'video_id', 'drop', 'watching_count',
       'video_duration', 'local_watching_time', 'video_progress_time',
       'video_start_time', 'video_end_time', 'local_start_time',
       'local_end_time', 'video_watching_duration', 'local_watching_duration',
       'video_is_end', 'video_is_start', 'video_is_start_end',
       'forward_seconds', 'local_watching_diff', 'speed'],
      dtype='object')

In [14]:
data.drop(['video_start_time', 'video_end_time', 'video_id'], axis=1, inplace=True)
gc.collect()

99

In [15]:
data['drop'].mean()

0.05742931269513214

In [16]:
data2 = data.groupby(uid2)['drop'].apply(list)
data2 = data2.reset_index()
data2['drop'] = data2['drop'].map(lambda x: 1 if 1 in x else 0)
gc.collect()

# video_start_time, video_end_time, local_start_time, local_end_time
tmp = data.groupby(uid2, as_index=False).agg({
    # 'watching_count': np.sum,
    # 'video_duration': np.sum,
    # 'local_watching_time': np.sum,
    # 'video_progress_time': np.sum,
    # 'video_start_time': np.min,
    # 'video_end_time': np.max,
    'local_start_time': np.min,
    'local_end_time': np.max,
    'video_is_end': np.sum,
    'video_is_start': np.mean,
    'video_is_start_end': np.sum,
    'forward_seconds': np.sum,
    'local_watching_diff': np.mean,
    'speed': np.mean
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [17]:
# watching_count
tmp = data.groupby(uid2, as_index=False)['watching_count'].agg({
    'watching_count_sum': 'sum',
    'watching_count_count': 'count',
    'watching_count_mean': 'mean',
    'watching_count_max': 'max',
    'watching_count_min': 'min',
    'watching_count_std': 'std',
    'watching_count_skew': 'skew',
    # 'watching_count_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [18]:
# video_duration
tmp = data.groupby(uid2, as_index=False)['video_duration'].agg({
    'video_duration_sum': 'sum',
    'video_duration_count': 'count',
    'video_duration_mean': 'mean',
    'video_duration_max': 'max',
    'video_duration_min': 'min',
    'video_duration_std': 'std',
    'video_duration_skew': 'skew',
    # 'video_duration_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [19]:
# local_watching_time
tmp = data.groupby(uid2, as_index=False)['local_watching_time'].agg({
    'local_watching_time_sum': 'sum',
    'local_watching_time_count': 'count',
    'local_watching_time_mean': 'mean',
    'local_watching_time_max': 'max',
    'local_watching_time_min': 'min',
    'local_watching_time_std': 'std',
    'local_watching_time_skew': 'skew',
    # 'local_watching_time_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [20]:
# video_progress_time
tmp = data.groupby(uid2, as_index=False)['video_progress_time'].agg({
    'video_progress_time_sum': 'sum',
    'video_progress_time_count': 'count',
    'video_progress_time_mean': 'mean',
    'video_progress_time_max': 'max',
    'video_progress_time_min': 'min',
    'video_progress_time_std': 'std',
    'video_progress_time_skew': 'skew',
    # 'video_progress_time_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [21]:
# 课程：观看结束时间-观看开始时间
data2['local_watching_duration'] = data2.apply(lambda x: (x['local_end_time'] - x['local_start_time']).seconds, axis=1)

data2['local_watching_diff'] = data2['local_watching_duration'] - data2['local_watching_time_sum']

In [22]:
data2.drop(['local_start_time', 'local_end_time'], axis=1, inplace=True)
del data
gc.collect()

20

In [23]:
data2['drop'].mean()

1.0

In [24]:
data2.columns

Index(['item_id', 'course_id', 'drop', 'video_is_end', 'video_is_start',
       'video_is_start_end', 'forward_seconds', 'local_watching_diff', 'speed',
       'watching_count_sum', 'watching_count_count', 'watching_count_mean',
       'watching_count_max', 'watching_count_min', 'watching_count_std',
       'watching_count_skew', 'video_duration_sum', 'video_duration_count',
       'video_duration_mean', 'video_duration_max', 'video_duration_min',
       'video_duration_std', 'video_duration_skew', 'local_watching_time_sum',
       'local_watching_time_count', 'local_watching_time_mean',
       'local_watching_time_max', 'local_watching_time_min',
       'local_watching_time_std', 'local_watching_time_skew',
       'video_progress_time_sum', 'video_progress_time_count',
       'video_progress_time_mean', 'video_progress_time_max',
       'video_progress_time_min', 'video_progress_time_std',
       'video_progress_time_skew', 'local_watching_duration'],
      dtype='object')

In [25]:
data2.to_csv('../../input/act_4_label.csv', index=False)
data2.to_hdf('../../input/act_4_label.h5', 'df', index=False)